#### 스타벅스 서울시 매장위치 크롤링

##### 스타벅스 웹사이트
- https://www.starbucks.co.kr/index.do

##### 셀레니움 등 라이브러리 임포트

In [49]:
!pip install tqdm

In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

In [69]:
# 크롬 웹드라이버 실행

driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [72]:
# 웹드라이버 선택할 위치값 작성(20년 8월과 동일)
btn_seoul = '#container > div > form > fieldset > div > section > article.find_store_cont > ' +\
            'article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > ' +\
            'li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_seoul).click()

In [74]:
# 서울 전체클릭
btn_all = '#mCSB_2_container > ul > li > a'
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [75]:
# 스타벅스는 한화면에 전부 리스트업되기 때문에 함수를 만들필요도 없음
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [ ]:
soup

In [102]:
# 검색결과 위치
starbuks_soup_list = soup.select('quickResultLstCon')
len(starbuks_soup_list)

0

In [ ]:
starbuks_soup_list[0]

In [92]:
#  매장정보 확인
starbuks_store = starbuks_soup_list[0]
name = starbuks_store.select('strong')[0].text.strip()
lat  = starbuks_store['data-lat'].strip()
long = starbuks_store['data-long'].strip()
store_type = starbuks_store.select('i')[0]['class'][0].split('_')[1] #_로 자르면 두번째 값이 general/reserve
address = starbuks_store.select('p')[0].get_text(separator='|').split('|')[0]

print(name) # 매장명
print(lat, long, sep=' / ') # 매장 지도 위치
print(store_type)
print(address)


역삼아레나빌딩
37.501087 / 127.043069
general
서울특별시 강남구 언주로 425 (역삼동)


In [108]:
# 600개 매장 데이터 가져오기
starbuks_seoul_list = []
starbuks_soup_list
for item in tqdm(starbuks_soup_list):
    name = item.select('strong')[0].text.strip()
    lat  = item['data-lat'].strip()
    long = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0].split('_')[1] #_로 자르면 두번째 값이 general/reserve
    address = item.select('p')[0].get_text(separator='|').split('|')[0] 

    starbuks_seoul_list.append([name, lat, long, store_type, address])


0it [00:00, ?it/s]


In [104]:
len(starbuks_seoul_list)

0

In [105]:
#DataFrame 생성
colums = ['매장명', '위도', '경도', '매장타입', '주소']
df= pd.DataFrame(starbuks_seoul_list, columns=colums)
df.tail()

,매장명,위도,경도,매장타입,주소
